In [1]:
import linopy
import pyomo 
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Secondary Frequency Regulation for BESS

. Read the frequency deviation file
. Read the activated secondary control power file
. Generate the associated activation signal for each time slot

### Data preparation

In [10]:
f_deviation = pd.read_csv("/srv/students/knguyen/Thesis/Operation/Code/Data/germany_2020_01.csv", index_col=0)


ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2


In [51]:
afrr_regulation_signal = pd.read_csv("/srv/students/knguyen/Thesis/Operation/Code/Data/Sekundaerregelleistung_2020.csv", delimiter =";", header =3, encoding='UTF-16 LE')

In [28]:

f_deviation.index=pd.to_datetime(f_deviation.index) #set the index to panda datetime format

testday =f_deviation.loc[(f_deviation.index.day==1)] #select Jan 1st as the test day
testday=testday.bfill() #fill NaN with the next value
testhorizon=testday.loc[:'2020-01-01 01:00:00']
testhorizon['15_min_interval'] = testhorizon.index.floor('15min')


In [52]:
afrr_regulation_signal.rename(columns ={"Datum":"Date"}, inplace=True)
afrr_regulation_signal.set_index(pd.to_datetime(afrr_regulation_signal[['Date','Von']].agg(' '.join, axis =1), dayfirst=True), inplace=True) 


In [ ]:
testday1 = afrr_regulation_signal.loc[(afrr_regulation_signal.index.day==1)]
testhorizon1 = testda1.loc[:'2020-01-01 01:00:00']

In [58]:
testhorizon

,Frequency,15_min_interval,Pbatt,SOC,DOD
2020-01-01 00:00:00,25.0,2020-01-01 00:00:00,0,0,0
2020-01-01 00:00:01,25.0,2020-01-01 00:00:00,0,0,0
2020-01-01 00:00:02,24.0,2020-01-01 00:00:00,0,0,0
2020-01-01 00:00:03,23.0,2020-01-01 00:00:00,0,0,0
2020-01-01 00:00:04,22.0,2020-01-01 00:00:00,0,0,0
...,...,...,...,...,...
2020-01-01 00:59:56,7.0,2020-01-01 00:45:00,0,0,0
2020-01-01 00:59:57,8.0,2020-01-01 00:45:00,0,0,0
2020-01-01 00:59:58,8.0,2020-01-01 00:45:00,0,0,0
2020-01-01 00:59:59,7.0,2020-01-01 00:45:00,0,0,0


### Simple battery model

In [39]:
Battery_Qmax = 4                     #MWh
Battery_Qmin = 0.2*Battery_Qmax
Battery_Pmax = 1                     #MW
Battery_Pop  = Battery_Pmax/2        #Maximum operation power of battery (max bidding)
Battery_iSOC  =50                         #assumed initial SOC is 50%
testhorizon['Pbatt']=0
testhorizon['SOC']  =0
testhorizon['DOD'] = 0

/tmp/ipykernel_3466641/1471125845.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testhorizon['Pbatt']=0
/tmp/ipykernel_3466641/1471125845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testhorizon['SOC']  =0
/tmp/ipykernel_3466641/1471125845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

### Operation Model

Assign activation signal for each 15min interval
calculate output power
Map output power to the testhorizon
calculate battery profile